# Importing Data from Yahoo Finance with yfinance

### Setup

In [3]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler

from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute

from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series


## Historical Price & Volume Data for one Stock

In [9]:
ticker = "NVDA"
NVDA = yf.download(ticker)
NVDA.info()

[*********************100%%**********************]  1 of 1 completed
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6288 entries, 1999-01-22 to 2024-01-18
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       6288 non-null   float64
 1   High       6288 non-null   float64
 2   Low        6288 non-null   float64
 3   Close      6288 non-null   float64
 4   Adj Close  6288 non-null   float64
 5   Volume     6288 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 343.9 KB


In [ ]:
NVDA = yf.download(ticker, start = "2014-01-01", end = "2018-12-31")
print(NVDA.info())
print(NVDA.head())
print(NVDA.tail())

### Setting specific Time Periods

In [ ]:
yf.download(ticker, period  = "ytd")

In [ ]:
yf.download(ticker, period  = "1mo")

In [ ]:
df = yf.download(ticker, period  = "5d")

## pandas-ta and ts-fresh.

The pandas-ta documentation is [here](https://twopirllc.github.io/pandas-ta/).

The ts-fresh documentation is [here](https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html)

In [ ]:
def ts_fresh_features(data):

    target = (data['Close'].shift() < data['Close']).astype(int)

    # Required for ts_fresh
    data['id'] = range(len(data))
    data['time'] = data.index

    df_rolled = roll_time_series(data, column_id="id", column_sort="time")

    extraction_settings = EfficientFCParameters()
    extracted_features = extract_features(df_rolled, column_id='id', column_sort='time',
                        default_fc_parameters=extraction_settings,
                        impute_function=impute, show_warnings=False)

    extracted_features.set_index(target.index, inplace=True)

    # impute(extracted_features)
    # features_filtered = select_features(extracted_features, target)

    return extracted_features, target

In [ ]:
# Define time series tickers
ticker = ["NVDA"]
index_tickers = ["^DJI", "^GSPC", "^IXIC"]

In [ ]:
pd.set_option('display.max_rows', 100)

data = yf.download(ticker, period  = "10y")

extracted_features, target = ts_fresh_features(data)
ts_fresh_df = extracted_features[['Volume__benford_correlation', \
                                  'Open__benford_correlation', 'High__benford_correlation']]

relevance_table_clf = calculate_relevance_table(extracted_features, target)
relevance_table_clf.sort_values("p_value", inplace=True)
relevance_table_clf

In [ ]:
data.head()

In [ ]:

# Volume weighted average price requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
# df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)

data = pd.DataFrame() # Empty DataFrame

 # OR if you have yfinance installed
data = yf.download(ticker, period  = "10y")
indexes = yf.download(index_tickers, period = "10y").Close
data = pd.concat([data, ts_fresh_df], axis=1)
print(data.shape)

# print(data.ta.indicators())

aberration_df = ta.aberration(data['High'], data['Low'], data['Close'], length=5, \
                                   atr_length=14)
data = pd.concat([data, aberration_df], axis=1)

bbands_df = ta.bbands(data['Close'], length=5, std=2, mamode='sma')
data = pd.concat([data, bbands_df], axis=1)

data['AO_5_34'] = ta.ao(data['High'], data['Low'], fast=5, slow=34)
data['BOP'] = ta.bop(data['Open'], data['High'], data['Low'], data['Close'])
data['MOM_2'] = ta.mom(data['Close'], length=2)
data['MOM_4'] = ta.mom(data['Close'], length=4)
data['MOM_6'] = ta.mom(data['Close'], length=6)
data['SMA_30'] = ta.sma(data['Close'], length=30)
data['SMA_60'] = ta.sma(data['Close'],length=60)
data['SMA_90'] = ta.sma(data['Close'],length=90)
data['RSI_14'] = ta.rsi(close=data['Close'], length=14)
# Volume indicators
data['CMF_20'] = ta.cmf(high=data['High'], low=data['Low'], close=data['Close'], \
                        volume=data['Volume'], length=20)
data['AD'] = ta.ad(high=data['High'], low=data['Low'], close=data['Close'], \
                        volume=data['Volume'])

# Calculate the stochastic oscillator
stoch = ta.stoch(high=data['High'], low=data['Low'], close=data['Close'], k=14, d=3, smooth_k=3)
# Add the stochastic oscillator values to the DataFrame
data['%K_14_3_3'] = stoch['STOCHk_14_3_3']
data['%D_14_3_3'] = stoch['STOCHd_14_3_3']

indexes['MOM_2_DJI'] = ta.mom(indexes['^DJI'], length=2)
indexes['MOM_4_DJI'] = ta.mom(indexes['^DJI'], length=4)
indexes['MOM_6_DJI'] = ta.mom(indexes['^DJI'], length=6)
indexes['SMA_30_DJI'] = ta.sma(indexes['^DJI'], length=30)
indexes['SMA_60_DJI'] = ta.sma(indexes['^DJI'],length=60)
indexes['SMA_90_DJI'] = ta.sma(indexes['^DJI'],length=90)
indexes['RSI_14_DJI'] = ta.rsi(close=indexes['^DJI'], length=14)

indexes['MOM_2_GSPC'] = ta.mom(indexes['^GSPC'], length=2)
indexes['MOM_4_GSPC'] = ta.mom(indexes['^GSPC'], length=4)
indexes['MOM_6_GSPC'] = ta.mom(indexes['^GSPC'], length=6)
indexes['SMA_30_GSPC'] = ta.sma(indexes['^GSPC'], length=30)
indexes['SMA_60_GSPC'] = ta.sma(indexes['^GSPC'],length=60)
indexes['SMA_90_GSPC'] = ta.sma(indexes['^GSPC'],length=90)
indexes['RSI_14_GSPC'] = ta.rsi(close=indexes['^GSPC'], length=14)

indexes['MOM_2_IXIC'] = ta.mom(indexes['^IXIC'], length=2)
indexes['MOM_4_IXIC'] = ta.mom(indexes['^IXIC'], length=4)
indexes['MOM_6_IXIC'] = ta.mom(indexes['^IXIC'], length=6)
indexes['SMA_30_IXIC'] = ta.sma(indexes['^IXIC'], length=30)
indexes['SMA_60_IXIC'] = ta.sma(indexes['^IXIC'],length=60)
indexes['SMA_90_IXIC'] = ta.sma(indexes['^IXIC'],length=90)
indexes['RSI_14_IXIC'] = ta.rsi(close=indexes['^IXIC'], length=14)


assert indexes.shape[0] == data.shape[0]
data = pd.concat([data, indexes], axis=1)
data = data.astype('float64')

data_filtered = data.iloc[100:]

# Initialize the StandardScaler
scaler = StandardScaler()

# Normalize all columns in the DataFrame using z-scaling
df_normalized = pd.DataFrame(scaler.fit_transform(data_filtered), columns=data_filtered.columns)


# Set the index of df2 to be the index of df1
df_normalized.set_index(data_filtered.index, inplace=True)


In [ ]:
df_normalized.index

In [ ]:

for i, c in zip(range(len(df_normalized.columns)), df_normalized.columns):
    print(f'{i}: {c}')


In [ ]:
df_normalized.isnull().sum()
df_normalized.count()
df_normalized.info()


In [ ]:
# Write the DataFrame to a CSV file
df_normalized['Name'] = 'NVDA'
df_normalized.to_csv('./data/data-2d/NVDA//NVDA.csv', index=True)

## Frequency Settings (Intraday)

In [ ]:
ticker = "GE"
# yf.download(ticker, period  = "1mo", interval = "1h")
# yf.download(ticker, period  = "1mo", interval = "30m")
# yf.download(ticker, period  = "1mo", interval = "5m")
# yf.download(ticker, period  = "1mo", interval = "1m")
# yf.download(ticker, period  = "5d", interval = "1m")

In [ ]:
GE = yf.download(ticker, period  = "5d", interval = "1m")
GE.head(10)

In [ ]:
GE.describe()

In [ ]:
yf.download(ticker, period  = "5d", prepost= True, interval = "30m")

## Stock Splits and Dividends

In [ ]:
ticker = "AAPL"
AAPL = yf.download(ticker, period  = "10y", actions = True)
AAPL.head()

In [ ]:
AAPL[AAPL["Dividends"] > 0]

In [ ]:
AAPL.loc["2019-08-05":"2019-08-15"]

In [ ]:
AAPL.loc["2019-08-05":"2019-08-15"].diff()

In [ ]:
AAPL[AAPL["Stock Splits"] > 0]

In [ ]:
AAPL.loc["2014-06-05":"2014-06-15"]

## Exporting to CSV/Excel

In [ ]:
ticker = "GE"

In [ ]:
GE = yf.download(ticker, period  = "5y")
GE.head()

In [ ]:
GE.to_csv("GE.csv")
pd.read_csv("GE.csv", parse_dates = ["Date"], index_col = "Date")

In [ ]:
GE.to_excel("GE.xls", engine = "openpyxl") # engine added
pd.read_excel("GE.xls", parse_dates= ["Date"], index_col= "Date")

## Importing Data for many Stocks

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
ticker = ["GE", "AAPL", "META"] 
yf.download(ticker, period  = "5y")
# yf.download(ticker, period  = "5y" , group_by= "Ticker")


In [ ]:
stocks = yf.download(ticker, period  = "5y").Close
stocks.head()

In [ ]:
stocks.plot()
plt.show()

## Importing Financial Indexes

In [6]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [7]:
ticker = ["^DJI", "^GSPC", "^IXIC"]
indexes = yf.download(ticker, period  = "1y")
indexes.head(10)

[*********************100%%**********************]  3 of 3 completed


Adj Close                                    Close  \
                    ^DJI        ^GSPC         ^IXIC          ^DJI   
Date                                                                
2023-01-18  33296.960938  3928.860107  10957.009766  33296.960938   
2023-01-19  33044.558594  3898.850098  10852.269531  33044.558594   
2023-01-20  33375.488281  3972.610107  11140.429688  33375.488281   
2023-01-23  33629.558594  4019.810059  11364.410156  33629.558594   
2023-01-24  33733.960938  4016.949951  11334.269531  33733.960938   
2023-01-25  33743.839844  4016.219971  11313.360352  33743.839844   
2023-01-26  33949.410156  4060.429932  11512.410156  33949.410156   
2023-01-27  33978.078125  4070.560059  11621.709961  33978.078125   
2023-01-30  33717.089844  4017.770020  11393.809570  33717.089844   
2023-01-31  34086.039062  4076.600098  11584.549805  34086.039062   

                                               High               \
                  ^GSPC         ^IXIC          ^DJI        ^GSPC   
Date                                                               
2023-01-18  3928.860107  10957.009766  34016.531250  4014.159912   
2023-01-19  3898.850098  10852.269531  33227.488281  3922.939941   
2023-01-20  3972.610107  11140.429688  33381.949219  3972.959961   
2023-01-23  4019.810059  11364.410156  33782.878906  4039.310059   
2023-01-24  4016.949951  11334.269531  33782.921875  4023.919922   
2023-01-25  4016.219971  11313.360352  33773.089844  4019.550049   
2023-01-26  4060.429932  11512.410156  33953.789062  4061.570068   
2023-01-27  4070.560059  11621.709961  34164.328125  4094.209961   
2023-01-30  4017.770020  11393.809570  34055.289062  4063.850098   
2023-01-31  4076.600098  11584.549805  34095.230469  4077.159912   

                                   Low                             \
                   ^IXIC          ^DJI        ^GSPC         ^IXIC   
Date                                                                
2023-01-18  11223.410156  33269.898438  3926.590088  10952.049805   
2023-01-19  10932.519531  32982.050781  3885.540039  10804.570312   
2023-01-20  11143.169922  32948.929688  3897.860107  10885.650391   
2023-01-23  11405.500000  33316.250000  3971.639893  11144.030273   
2023-01-24  11378.150391  33310.558594  3989.790039  11282.650391   
2023-01-25  11334.219727  33273.210938  3949.060059  11069.179688   
2023-01-26  11516.049805  33635.601562  4013.290039  11341.190430   
2023-01-27  11691.889648  33830.851562  4048.699951  11470.269531   
2023-01-30  11553.309570  33695.179688  4015.550049  11388.540039   
2023-01-31  11586.009766  33664.910156  4020.439941  11398.370117   

                    Open                                Volume              \
                    ^DJI        ^GSPC         ^IXIC       ^DJI       ^GSPC   
Date                                                                         
2023-01-18  33948.488281  4002.250000  11170.950195  351670000  4298710000   
2023-01-19  33171.351562  3911.840088  10890.400391  319770000  3991500000   
2023-01-20  33073.460938  3909.040039  10924.660156  367330000  4013360000   
2023-01-23  33439.558594  3978.139893  11171.940430  355220000  3945210000   
2023-01-24  33444.718750  4001.739990  11302.929688  299310000  3320430000   
2023-01-25  33538.359375  3982.709961  11146.530273  341720000  3724020000   
2023-01-26  33771.660156  4036.080078  11458.400391  327000000  3809590000   
2023-01-27  33952.929688  4053.719971  11470.469727  441260000  3907760000   
2023-01-30  33909.210938  4049.270020  11512.339844  343920000  3802000000   
2023-01-31  33803.558594  4020.850098  11398.580078  360240000  4679320000   

                        
                 ^IXIC  
Date                    
2023-01-18  5288630000  
2023-01-19  4660800000  
2023-01-20  5980110000  
2023-01-23  5997810000  
2023-01-24  5670860000  
2023-01-25  5485570000  
2023-01-26  5732890000  
2023-01-27  6243800000  
2023-01-30  5122320000  
2023-01-31  5695150

In [ ]:
norm = indexes.div(indexes.iloc[0]).mul(100)
norm.head()

In [ ]:
norm.plot()
plt.show()

In [ ]:
ticker = ["^DJITR", "^SP500TR"]
indexes = yf.download(ticker, period  = "5y").Close
indexes

## Importing Currencies / FX

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
ticker = "EURUSD=X"
ticker2 = "USDEUR=X"
ticker3 = "USDGBP=X"

In [ ]:
df = yf.download(ticker, period  = "5y")
df.head()

## Importing Cryptocurrencies

In [ ]:
import yfinance as yf

# Get the tickers for available cryptocurrencies
cryptocurrencies = yf.Tickers

In [ ]:
ticker = ["BTC-EUR", "ETH-USD"]

In [ ]:
crypt = yf.download(ticker, period  = "5y").Close
crypt.head()

In [ ]:
crypt.plot()
plt.show()

## Importing Funds & ETFs

In [ ]:
ticker = "TLT" # iShares 20+ Year Treasury Bond ETF
ticker2 = "OMOIX" # Vivaldi Multi-Strategy Fund Class I

In [ ]:
yf.download(ticker, period  = "5y")

In [ ]:
yf.download(ticker2, period  = "5y") 

## Importing Treasury Yields

In [ ]:
ticker1 = "^TNX" # 10Y Tresury Yield
ticker2 = "^FVX" # 5Y Treasury Yield

In [ ]:
ticker = ["^TNX", "^FVX"]

In [ ]:
yf.download(ticker, period  = "5y")

In [ ]:
yields = yf.download(ticker, period  = "5y").Close

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
yields.plot()
plt.show()

In [ ]:
ticker = "DIS"

## Stock Fundamentals, Meta Data and Performance Metrics

Note: __yfinance is currently unstable/unreliable__ when it comes to downloading fundamental data with the ticker object. In particular, __ticker.info()__ is flawed.

__Action required__: Check for the __latest yfinance versions__ and update with the following command (Anaconda Prompt / Terminal):

pip install yfinance --upgrade

In the following, I have added an __alternative Yahoo Finance API Wrapper__, which is more stable/reliable: __yahooquery__

__Action required: Please install yahooquery with the following command (Anaconda Prompt / Terminal):__

pip install yahooquery

__yfinance__ (currently not working, please check again with latest yfinance version)

In [4]:
ticker = "DIS"
dis = yf.Ticker(ticker)
dis.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-12-20 00:00:00-05:00,93.239998,94.269997,91.250000,91.269997,13060600,0.0,0.0
2023-12-21 00:00:00-05:00,91.930000,92.080002,91.059998,92.019997,13037200,0.0,0.0
2023-12-22 00:00:00-05:00,91.620003,92.239998,90.750000,91.019997,9411500,0.0,0.0
2023-12-26 00:00:00-05:00,90.839996,91.269997,90.599998,90.949997,8086900,0.0,0.0
2023-12-27 00:00:00-05:00,90.910004,90.940002,90.040001,90.379997,11123500,0.0,0.0
2023-12-28 00:00:00-05:00,90.089996,90.980003,89.970001,90.400002,8479600,0.0,0.0
2023-12-29 00:00:00-05:00,90.120003,90.599998,89.860001,90.290001,9201300,0.0,0.0
2024-01-02 00:00:00-05:00,90.099998,91.480003,89.730003,90.709999,10587600,0.0,0.0
2024-01-03 00:00:00-05:00,90.230003,92.080002,90.000000,91.650002,11929800,0.0,0.0


In [5]:
dis.info # currently not working

{'address1': '500 South Buena Vista Street',
 'city': 'Burbank',
 'state': 'CA',
 'zip': '91521',
 'country': 'United States',
 'phone': '818 560 1000',
 'website': 'https://www.thewaltdisneycompany.com',
 'industry': 'Entertainment',
 'industryKey': 'entertainment',
 'industryDisp': 'Entertainment',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'The Walt Disney Company operates as an entertainment company worldwide. It operates through three segments: Entertainment, Sports, and Experiences. The company produces and distributes film and television video streaming content under the ABC Television Network, Disney, Freeform, FX, Fox, National Geographic, and Star brand television channels, as well as ABC television stations and A+E television networks; and produces original content under the ABC Signature, Disney Branded Television, FX Productions, Lucasfilm, Marvel, National Geographic Studios

In [6]:
pd.Series(dis.info)

address1             500 South Buena Vista Street
city                                      Burbank
state                                          CA
zip                                         91521
country                             United States
                                 ...             
grossMargins                              0.33406
ebitdaMargins                             0.16537
operatingMargins                          0.10197
financialCurrency                             USD
trailingPegRatio                           0.4559
Length: 131, dtype: object

In [7]:
df = pd.Series(dis.info, name = "DIS").to_frame().T
df

,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,...,returnOnEquity,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio
DIS,500 South Buena Vista Street,Burbank,CA,91521,United States,818 560 1000,https://www.thewaltdisneycompany.com,Entertainment,entertainment,Entertainment,...,0.03062,5724375040,9866000384,0.623,0.054,0.33406,0.16537,0.10197,USD,0.4559


In [8]:
ticker = ["MSFT", "META"] # FB is now META!

In [9]:
for i in ticker:
    df.loc["{}".format(i)] = pd.Series(yf.Ticker(i).info)    

In [10]:
df

,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,...,returnOnEquity,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio
DIS,500 South Buena Vista Street,Burbank,CA,91521,United States,818 560 1000,https://www.thewaltdisneycompany.com,Entertainment,entertainment,Entertainment,...,0.03062,5724375040,9866000384,0.623,0.054,0.33406,0.16537,0.10197,USD,0.4559
MSFT,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,software-infrastructure,Software - Infrastructure,...,0.39107,51013873664,94966996992,0.272,0.128,0.69511,0.5015,0.47587,USD,2.3494
META,1601 Willow Road,Menlo Park,CA,94025,United States,650 543 4800,https://investor.fb.com,Internet Content & Information,internet-content-information,Internet Content & Information,...,0.22275,29440124928,66219999232,1.679,0.232,0.80115,0.42583,0.41375,USD,0.8725


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, DIS to META
Columns: 131 entries, address1 to trailingPegRatio
dtypes: object(131)
memory usage: 3.1+ KB


__NEW__ (Jan 2023): fast_info | Update: fast_info changed to get_fast_info 

In [12]:
fast_info = dis.get_fast_info # Updated
fast_info()

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']

In [ ]:
for key in fast_info():
    print(f'{key}: {fast_info()[key]}') 

__yahooquery__

In [13]:
from yahooquery import Ticker

In [19]:
yq = Ticker(symbols = ticker)  
yq.summary_detail
yq.balance_sheet().shape


(7, 85)

In [35]:
yq.cash_flow()

,asOfDate,periodType,currencyCode,AssetImpairmentCharge,BeginningCashPosition,CapitalExpenditure,CashDividendsPaid,CashFlowFromContinuingFinancingActivities,CashFlowFromContinuingInvestingActivities,CashFlowFromContinuingOperatingActivities,...,OtherNonCashItems,PurchaseOfBusiness,PurchaseOfInvestment,PurchaseOfPPE,RepaymentOfDebt,RepurchaseOfCapitalStock,SaleOfInvestment,SaleOfPPE,StockBasedCompensation,UnrealizedGainLossOnInvestmentSecurities
symbol,,,,,,,,,,,,,,,,,,,,,
META,2020-12-31,12M,USD,0.000000e+00,1.927900e+10,-1.516300e+10,NaN,-1.029200e+10,-3.005900e+10,3.874700e+10,...,8.500000e+07,-3.880000e+08,-4.029100e+10,-1.516300e+10,-6.040000e+08,-6.272000e+09,2.577100e+10,48000000.0,6.536000e+09,33000000.0
META,2021-12-31,12M,USD,0.000000e+00,1.795400e+10,-1.869000e+10,NaN,-5.072800e+10,-7.570000e+09,5.768300e+10,...,1.050000e+08,-8.510000e+08,-3.045400e+10,-1.869000e+10,-6.770000e+08,-4.453700e+10,4.258600e+10,123000000.0,9.164000e+09,-232000000.0
META,2022-12-31,12M,USD,2.218000e+09,1.686500e+10,-3.143100e+10,NaN,-2.213600e+10,-2.897000e+10,5.047500e+10,...,1.519000e+09,-1.312000e+09,-9.631000e+09,-3.143100e+10,-8.500000e+08,-2.795600e+10,1.315800e+10,245000000.0,1.199200e+10,463000000.0
META,2023-09-30,TTM,USD,3.147000e+09,1.525100e+10,-2.864400e+10,NaN,-1.815900e+10,-2.555400e+10,6.622000e+10,...,1.726000e+09,-6.270000e+08,-2.556000e+09,-2.864400e+10,-9.860000e+08,-2.069500e+10,6.088000e+09,203000000.0,1.361100e+10,NaN
MSFT,2020-06-30,12M,USD,NaN,1.135600e+10,-1.544100e+10,-1.513700e+10,-4.603100e+10,-1.222300e+10,6.067500e+10,...,NaN,-2.521000e+09,-7.719000e+10,-1.544100e+10,-5.518000e+09,-2.296800e+10,8.417000e+10,NaN,5.289000e+09,NaN
MSFT,2021-06-30,12M,USD,NaN,1.357600e+10,-2.062200e+10,-1.652100e+10,-4.848600e+10,-2.757700e+10,7.674000e+10,...,NaN,-8.909000e+09,-6.292400e+10,-2.062200e+10,-3.750000e+09,-2.738500e+10,6.580000e+10,NaN,6.118000e+09,NaN
MSFT,2022-06-30,12M,USD,NaN,1.422400e+10,-2.388600e+10,-1.813500e+10,-5.887600e+10,-3.031100e+10,8.903500e+10,...,NaN,-2.203800e+10,-2.645600e+10,-2.388600e+10,-9.023000e+09,-3.269600e+10,4.489400e+10,NaN,7.502000e+09,NaN
MSFT,2023-06-30,12M,USD,NaN,1.393100e+10,-2.810700e+10,-1.980000e+10,-4.393500e+10,-2.268000e+10,8.758200e+10,...,NaN,-1.670000e+09,-3.765100e+10,-2.810700e+10,-2.750000e+09,-2.224500e+10,4.786400e+10,NaN,9.611000e+09,NaN
MSFT,2023-09-30,TTM,USD,NaN,2.288400e+10,-3.174100e+10,-2.023000e+10,-1.829100e+10,-1.904500e+10,9.496700e+10,...,NaN,-2.507000e+09,-4.109800e+10,-3.174100e+10,-3.250000e+09,-2.150300e+10,5.953900e+10,NaN,9.926000e+09,NaN


In [23]:
yq.earning_history

maxAge  epsActual  epsEstimate  epsDifference  surprisePercent  \
symbol row                                                                   
MSFT   0         1       2.32         2.29           0.03            0.013   
       1         1       2.45         2.23           0.22            0.099   
       2         1       2.69         2.55           0.14            0.055   
       3         1       2.73         2.42           0.31            0.128   
META   0         1       1.76         2.22          -0.46           -0.207   
       1         1       2.20         2.03           0.17            0.084   
       2         1       2.98         2.91           0.07            0.024   
       3         1       4.00         3.31           0.69            0.208   

               quarter period  
symbol row                     
MSFT   0    2022-12-31    -4q  
       1    2023-03-31    -3q  
       2    2023-06-30    -2q  
       3    2023-09-30    -1q  
META   0    2022-12-31    -4q  
       1    2023-03-31    -3q  
       2    2023-06-30    -2q  
       3    2023-09-30    -1q

In [21]:
yq.key_stats['MSFT']

{'maxAge': 1,
 'priceHint': 2,
 'enterpriseValue': 2889081159680,
 'forwardPE': 30.78533,
 'profitMargins': 0.35314998,
 'floatShares': 7425796261,
 'sharesOutstanding': 7432260096,
 'sharesShort': 51394982,
 'sharesShortPriorMonth': 49279547,
 'sharesShortPreviousMonthDate': '2023-11-29 16:00:00',
 'dateShortInterest': '2023-12-28 16:00:00',
 'sharesPercentSharesOut': 0.0069,
 'heldPercentInsiders': 0.00055,
 'heldPercentInstitutions': 0.73023003,
 'shortRatio': 1.94,
 'shortPercentOfFloat': 0.0069,
 'beta': 0.876,
 'impliedSharesOutstanding': 7432260096,
 'category': None,
 'bookValue': 29.702,
 'priceToBook': 13.422329,
 'fundFamily': None,
 'legalType': None,
 'lastFiscalYearEnd': '2023-06-29 17:00:00',
 'nextFiscalYearEnd': '2024-06-29 17:00:00',
 'mostRecentQuarter': '2023-09-29 17:00:00',
 'earningsQuarterlyGrowth': 0.27,
 'netIncomeToCommon': 77096001536,
 'trailingEps': 10.32,
 'forwardEps': 12.95,
 'pegRatio': 2.27,
 'lastSplitFactor': '2:1',
 'lastSplitDate': '2003-02-17 16:

In [22]:
yq.financial_data['MSFT'] # financial data

{'maxAge': 86400,
 'currentPrice': 398.67,
 'targetHighPrice': 480.0,
 'targetLowPrice': 298.1,
 'targetMeanPrice': 419.73,
 'targetMedianPrice': 420.5,
 'recommendationMean': 1.7,
 'recommendationKey': 'buy',
 'numberOfAnalystOpinions': 44,
 'totalCash': 143945007104,
 'totalCashPerShare': 19.368,
 'ebitda': 109482999808,
 'totalDebt': 105681002496,
 'quickRatio': 1.531,
 'currentRatio': 1.663,
 'totalRevenue': 218310000640,
 'debtToEquity': 47.881,
 'revenuePerShare': 29.347,
 'returnOnAssets': 0.14718,
 'returnOnEquity': 0.39106998,
 'freeCashflow': 51013873664,
 'operatingCashflow': 94966996992,
 'earningsGrowth': 0.272,
 'revenueGrowth': 0.128,
 'grossMargins': 0.69511,
 'ebitdaMargins': 0.5015,
 'operatingMargins': 0.47587,
 'profitMargins': 0.35314998,
 'financialCurrency': 'USD'}

In [ ]:
yq.summary_detail['MSFT'] # summary statistics

In [ ]:
df = pd.Series(yq.key_stats['MSFT']).to_frame().T
df

In [ ]:
df.index = ['MSFT']
df

In [ ]:
ticker = ["MSFT", "META"] # FB is now META!

In [ ]:
for i in ticker:
    df.loc["{}".format(i)] = pd.Series(Ticker(symbols = i).key_stats[i]) 

In [ ]:
df

## Importing Financials (Balance Sheet, P&L, Cashflows)

__yfinance__ (currently not working, please check again with latest yfinance version)

In [39]:
dis
type(dis)

yfinance.ticker.Ticker

In [40]:
dis.balance_sheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,19000000.0,19000000.0,19000000.0,19000000.0
Ordinary Shares Number,1830000000.0,1781000000.0,1818000000.0,1807063365.0
Share Issued,1849000000.0,1800000000.0,1837000000.0,1826063365.0
Net Debt,32249000000.0,36754000000.0,38447000000.0,40714000000.0
Total Debt,49895000000.0,51608000000.0,57635000000.0,61539000000.0
...,...,...,...,...
Receivables Adjustments Allowances,-115000000.0,-158000000.0,-170000000.0,-426000000.0
Other Receivables,2266000000.0,1999000000.0,2360000000.0,1835000000.0
Accounts Receivable,10179000000.0,10811000000.0,11177000000.0,11299000000.0
Cash Cash Equivalents And Short Term Investments,14182000000.0,11615000000.0,15959000000.0,17914000000.0


In [41]:
dis.income_stmt # income_stmt instead of financials!

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,-1075947000.0,-295200000.0,-6540000.0,-1548450000.0
Tax Rate For Calcs,0.289,0.328,0.01,0.27
Normalized EBITDA,15834000000.0,12897000000.0,9732000000.0,10828000000.0
Total Unusual Items,-3723000000.0,-900000000.0,-654000000.0,-5735000000.0
Total Unusual Items Excluding Goodwill,-3723000000.0,-900000000.0,-654000000.0,-5735000000.0
Net Income From Continuing Operation Net Minority Interest,2354000000.0,3193000000.0,2024000000.0,-2832000000.0
Reconciled Depreciation,5369000000.0,5163000000.0,5111000000.0,5345000000.0
Reconciled Cost Of Revenue,59201000000.0,54401000000.0,45131000000.0,43880000000.0
EBITDA,12111000000.0,11997000000.0,9078000000.0,5093000000.0
EBIT,6742000000.0,6834000000.0,3967000000.0,-252000000.0


In [42]:
dis.cashflow

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,4897000000.0,1067000000.0,1989000000.0,3596000000.0
Repurchase Of Capital Stock,NaN,NaN,NaN,0.0
Repayment Of Debt,-1675000000.0,-4016000000.0,-3737000000.0,-3533000000.0
Issuance Of Debt,83000000.0,333000000.0,64000000.0,18120000000.0
Capital Expenditure,-4969000000.0,-4943000000.0,-3578000000.0,-4022000000.0
...,...,...,...,...
Operating Gains Losses,-944000000.0,518000000.0,-277000000.0,-1024000000.0
Pension And Employee Benefit Expense,4000000.0,620000000.0,816000000.0,547000000.0
Earnings Losses From Equity Investments,-782000000.0,-816000000.0,-761000000.0,-651000000.0
Gain Loss On Investment Securities,-166000000.0,714000000.0,-332000000.0,-920000000.0


In [ ]:
ticker = ["AAPL", "META", "AMZN"]

In [ ]:
for i in ticker:
    yf.Ticker(i).income_stmt.to_csv("{}.csv".format(i))

__yahooquery__

In [ ]:
yq

In [ ]:
yq.balance_sheet().T 

In [ ]:
yq.income_statement().T

In [ ]:
yq.cash_flow().T

In [ ]:
ticker = ["AAPL", "META", "AMZN"]

In [ ]:
for i in ticker:
    Ticker(i).income_statement().T.to_csv("{}.csv".format(i))

## Importing Put / Call Options (Stocks)

In [ ]:
dis

In [ ]:
dis.option_chain()

In [ ]:
calls = dis.option_chain()[0]
calls

In [ ]:
puts = dis.option_chain()[1]
puts

## Streaming real-time Data

In [ ]:
import pandas as pd
import yfinance as yf
import time

In [ ]:
yf.download("EURUSD=X", interval = "1m", period = "1d").tail()

In [ ]:
data = yf.download("EURUSD=X", interval = "1m", period = "1d")

In [ ]:
print(data.index[-1], data.iloc[-1, 3])

In [ ]:
while True:
    time.sleep(5)
    data = yf.download("EURUSD=X", interval = "1m", period = "1d", progress = False)
    print(data.index[-1], data.iloc[-1, 3])

In [ ]:
while True:
    time.sleep(60)
    data = yf.download("EURUSD=X", interval = "1m", period = "1d", progress = False)
    print(data.index[-1], data.iloc[-1, 3])

url = "https://finance.yahoo.com/quote/SMCI/cash-flow?p=SMCI#:~:text=Quarterly-,Download,-Expand%20All"

In [38]:
import requests
url = "https://finance.yahoo.com/quote/SMCI/cash-flow?p=SMCI"
response = requests.get(url)
print(response)

with open('file.py', 'wb') as file:
    file.write(response.content)

<Response [404]>
